In [1]:
from weather import *
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from os import mkdir
from os.path import isdir, isfile
from accidents_montreal import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import Row
import math
from io import StringIO

In [2]:
fetch_accidents_montreal()
spark = (SparkSession
            .builder
            .appName("Road accidents prediction")
            .getOrCreate())
df=extract_accidents_montreal_dataframe(spark)
test=df.take(5)
test = list(map(lambda e: (e.LOC_LAT, e.LOC_LONG), test))
t= test[0]
lat, long = t
year, month, day = (2006,5,2)
hour = 0
get_weather(lat, long, year, month, day, hour)

Skip fetching montreal accidents dataset: already downloaded
Skip extraction of accidents montreal dataframe: already done, reading from file
['Dew Point Temp (°C)', 'Rel Hum (%)', 'Wind Dir (10s deg)', 'Wind Dir Flag', 'Wind Spd (km/h)', 'Wind Spd Flag', 'Visibility (km)', 'Visibility Flag', 'Stn Press (kPa)', 'Hmdx', 'Hmdx Flag', 'Wind Chill', 'Wind Chill Flag']


['Temp Flag', 'Dew Point Temp Flag', 'Rel Hum Flag', 'Stn Press Flag']


Row(Dew Point Temp (°C)=0.55, Dew Point Temp Flag=['M'], Hmdx=nan, Hmdx Flag=nan, Rel Hum (%)=37.75, Rel Hum Flag=['M'], Stn Press (kPa)=101.96, Stn Press Flag=['M'], Temp (°C)=14.956627338772165, Temp Flag=['M'], Visibility (km)=20.0, Visibility Flag=nan, Weather=['Rain', 'Cloudy'], Wind Chill=nan, Wind Chill Flag=nan, Wind Dir (10s deg)=8.833333333333334, Wind Dir Flag=nan, Wind Spd (km/h)=16.166666666666668, Wind Spd Flag=nan)

In [3]:
df.take(1)


[Row(NO_SEQ_COLL='SPVM _ 2015 _ 18203', JR_SEMN_ACCDN='VE', DT_ACCDN='2015/01/09', CD_MUNCP=66023.0, NO_CIVIQ_ACCDN=nan, SFX_NO_CIVIQ_ACCDN='NaN', BORNE_KM_ACCDN=nan, RUE_ACCDN='BELLECHASSE', TP_REPRR_ACCDN=1.0, ACCDN_PRES_DE='1RE AV', NB_METRE_DIST_ACCD=nan, CD_GENRE_ACCDN=40.0, CD_SIT_PRTCE_ACCDN=nan, CD_ETAT_SURFC=16.0, CD_ECLRM=1.0, CD_ENVRN_ACCDN=2.0, NO_ROUTE=nan, CD_CATEG_ROUTE=14.0, CD_ETAT_CHASS=nan, CD_ASPCT_ROUTE=11.0, CD_LOCLN_ACCDN=32.0, CD_POSI_ACCDN=nan, CD_CONFG_ROUTE=2.0, CD_ZON_TRAVX_ROUTR=nan, CD_PNT_CDRNL_ROUTE='NaN', CD_PNT_CDRNL_REPRR='NaN', CD_COND_METEO=12.0, NB_VEH_IMPLIQUES_ACCDN=2, NB_MORTS=0, NB_BLESSES_GRAVES=0, NB_BLESSES_LEGERS=0, HEURE_ACCDN='12:00:00-12:59:00', AN=2015, NB_VICTIMES_TOTAL=0, GRAVITE='Dommages matériels seulement', REG_ADM='Montréal(06)', MRC='Montréal (66 )', nb_automobile_camion_leger=2, nb_camionLourd_tractRoutier=0, nb_outil_equipement=0, nb_tous_autobus_minibus=0, nb_bicyclette=0, nb_cyclomoteur=0, nb_motocyclette=0, nb_taxi=0, nb_ur

# New for testing weather

# old for development and experimentation

In [ ]:
t= test[0]
lat, long = t
year, month, day = (2006,5,2)
hour = 0
stations = get_stations(lat, long, year, month, day)
weighted_average_num = 0
weighted_average_denum = 0
weathers = list()
cols=list()
for station in stations:
    identity=station[0]
    cache_file_path = f'data/weather/s{identity}_{year}_{month}.h5'
    if isfile(cache_file_path):
        df = pd.read_hdf(cache_file_path, key='w')
    else:
        url = (f'http://climate.weather.gc.ca/climate_data/bulk_data_e.html?'
               f'format=csv&stationID={identity}&Year={year}&Month={month}&Day={day}&'
               f'timeframe=1&submit=Download+Data')
        print(url)
        csvfile = get(url).text
        with StringIO(csvfile) as csvfile:
            skip_header(csvfile)

            df = pd.read_csv(csvfile,
                             index_col='Date/Time', parse_dates=['Date/Time'])

        if not isdir('data/weather/'):
            mkdir('data/weather/')
        df.to_hdf(cache_file_path, key='w')
    
    s=df.loc[f'{year}-{month}-{day} {hour}:00'] 
    s.loc["station_denom"] = station[1]
    """s['Temp (°C)'] = s.at['Temp (°C)']/station[1]
    print(s['Temp (°C)'])"""
    if len(cols) == 0:
        cols = df.columns.values.tolist() + ["station_denom"]
    weathers.append(s)

weathers = pd.DataFrame(weathers, columns=cols)


In [ ]:

temperatures = weathers[['Temp (°C)', 'station_denom']] \
    .dropna() \
    .apply(lambda row: pd.Series([row[0]/row[1], 1/row[1]]), axis=1) 
temperatures[0].sum() / temperatures[1].sum()

In [ ]:
numeric_cols=[col for col in list(wheathers.dtypes.index) if wheathers.dtypes[col] in ['int64','float64']]
non_numeric_cols = [col for col in list(wheathers.dtypes.index) if col not in numeric_cols+['Weather']]
numeric_wheathers = wheathers.loc[:,numeric_cols]
means = numeric_wheathers.mean()
print(means)

In [ ]:
non_num_wheathers = wheathers.loc[:,non_numeric_cols] 
weather = wheathers.loc[:,'Weather'].dropna().drop_duplicates().values.tolist()
non_num_wheathers.iat[5,2]='A'

def get_majority_vote(col):
    vals = col.dropna().drop_duplicates().tolist()
    if len(vals) > 0:
        e= max(set(vals), key=vals.count)
        return [v for v in vals if vals.count(e)==vals.count(v)]
    else:
        return ''
    
non_num_wheathers = non_num_wheathers.apply(lambda col:small_func(col), axis=0)
a=non_num_wheathers.index.values.tolist()
b=non_num_wheathers.values.tolist()
c=means.index.values.tolist()
d=means.values.tolist()
indexes = a + c + ['Weather']
vals = b + d + [weather]

In [ ]:
Row(**dict(zip(indexes, vals)))